In [1]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.signal as signal
import math
import matplotlib.pyplot as plt

In [2]:
def vibration_modes(K, M):
    w2, Phi = la.eigh(K, M)
    omega = np.sqrt(np.abs(w2))
    frequencies = omega / (2 * math.pi)
    return omega, frequencies, Phi


def compute_damping_matrix(omega, Phi, zeta):
    omega = np.diagflat(omega)
    zeta = np.diagflat(zeta)
    Cd_modal = 2 * (omega @ zeta)
    Cd = la.inv(Phi.T) @ (Cd_modal @ la.inv(Phi))
    return Cd


def vibration_response_statespace(M, K, Cd, dof, X0, T, F):
    A = np.zeros((2 * dof, 2 * dof))
    A[0:dof, dof : 2 * dof] = np.eye(dof)
    A[dof : 2 * dof, 0:dof] = -(la.inv(M)) @ K
    A[dof : 2 * dof, dof : 2 * dof] = -(la.inv(M)) @ Cd

    B = np.zeros((2 * dof, dof))
    B[dof : 2 * dof, :] = la.inv(M)

    C = np.zeros((dof, 2 * dof))
    C[:, 0:dof] = -(la.inv(M)) @ K
    C[:, dof : 2 * dof] = -(la.inv(M)) @ Cd

    D = la.inv(M)

    sys = signal.StateSpace(A, B, C, D)
    T, acceleration, X = signal.lsim(sys, F, T, X0)
    displacement = X[:, 0:dof]
    velocity = X[:, dof : 2 * dof]

    return acceleration, velocity, displacement


In [3]:
dof = 8
zeta = np.array([[0.05] * dof])  # Considering 5% modal damping ratios for all modes
time_step = 0.001
end_time = 100.0  # Simulation time = 100 sec
T = np.arange(0.0, end_time + time_step, time_step)  # Time-Steps Array
X0 = np.zeros(
    2 * dof
)  # Initial contidions {Dispalcement = X0[:, 0 : dof], Velocity = X0[:, dof : 2*dof]}
num_time_histories = 100  # Number of time-histories


In [4]:
# Load Force-time histories

force_df = pd.read_csv("../force_dataframe.csv")


In [5]:
# State #4 - Damaged (50% stiffness reduction in spring 3 and 5)

m_1 = 0.4900
m_2 = m_3 = m_4 = m_5 = m_6 = m_7 = m_8 = 0.4652

k_1 = k_2 = k_4 = k_6 = k_7 = k = 5500.0
k_3 = k_5 = 0.50 * k

M = np.array(
    [
        [m_1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, m_2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, m_3, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, m_4, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, m_5, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, m_6, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, m_7, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, m_8],
    ]
)

K4 = np.array(
    [
        [k_1, -k_1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [-k_1, k_1 + k_2, -k_2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, -k_2, k_2 + k_3, -k_3, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, -k_3, k_3 + k_4, -k_4, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, -k_4, k_4 + k_5, -k_5, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, -k_5, k_5 + k_6, -k_6, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, -k_6, k_6 + k_7, -k_7],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k_7, k_7],
    ]
)

omega4, frequencies4, Phi4 = vibration_modes(K4, M)
Cd4 = compute_damping_matrix(omega4, Phi4, zeta)


In [6]:
modal_properties = pd.DataFrame(Phi4, columns = [f"mode {j}" for j in range(1, dof + 1)])
modal_properties.insert(0, "Frequencies", frequencies4)
modal_properties.to_csv("state4_modal_properties.csv", index = False)

In [7]:
# Acceleration, Velocity and Displacement responses

for i in range(1, num_time_histories + 1):
    F = np.zeros((len(T), dof))
    F[:, 0] = force_df[f"F{i}"]
    acceleration4, velocity4, displacement4 = vibration_response_statespace(
        M, K4, Cd4, dof, X0, T, F
    )
    columns = [f"m{j}" for j in range(1, dof + 1)]

    acceleration4_df = pd.DataFrame(acceleration4, columns=columns)
    acceleration4_df.insert(0, "Time", T)
    acceleration4_df.to_csv(f"state4_acceleration_timehistory{i}.csv", index=False)

    velocity4_df = pd.DataFrame(velocity4, columns=columns)
    velocity4_df.insert(0, "Time", T)
    velocity4_df.to_csv(f"state4_velocity_timehistory{i}.csv", index=False)

    displacement4_df = pd.DataFrame(displacement4, columns=columns)
    displacement4_df.insert(0, "Time", T)
    displacement4_df.to_csv(f"state4_displacement_timehistory{i}.csv", index=False)
